# Hackathon Query Notebook

Welcome to the hackathon! This notebook is your starting point for querying the main dataset.

It's designed to connect to a **remote Tentris server** that is already running and loaded with all the data. You do not need to run or install Tentris yourself.

**Your only task:** Follow the setup steps, run the cells, and start writing your own queries!

### Step 1: Environment Setup with `uv` (One-Time Setup)

This notebook uses [`uv`](https://github.com/astral-sh/uv) for fast and simple Python environment setup. `uv` is a modern replacement for `venv` and `pip`.

**1. Install `uv` (from your terminal):**
Before you can run the cell below, you need to install `uv` on your system. Run **one** of the following commands in your regular terminal (not in the notebook).

*On macOS / Linux:*
```bash
curl -LsSf [https://astral.sh/uv/install.sh](https://astral.sh/uv/install.sh) | sh
```

*On Windows (PowerShell) or if the above fails:*
```bash
# You may need to run PowerShell as Administrator
pip install uv
```
*(You may need to restart VS Code for it to find the new `uv` command)*

**2. Run this cell to create the environment:**
The cell below will use `uv` to:
1.  Create a local virtual environment in a folder named `.venv`.
2.  Install all the packages defined in `pyproject.toml` (including `ipykernel`).

**3. Set your VS Code Kernel:**
When running the cell, click "Select Kernel" in the top-right and choose create new virtual env - you will use this root-level .venv only to run the first cell. After that, the newly created .venv will be used.

In [4]:
# 1. Clean up any old environment
!rm -rf .venv

# 2. Create a new, empty virtual environment
!uv venv

# 3. Install ipykernel so Jupyter can use this venv
!uv add --dev ipykernel


# 4. Sync your project dependencies from the toml file
!uv pip sync ../tentris-quickstart/pyproject.toml

# 5. (NEW) Register this venv as a Jupyter kernel
!.venv/bin/python -m ipykernel install --user --name="tentris-env" --display-name="Python (Tentris)"


Using CPython 3.13.2
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 41 packages in 1ms
Installed 36 packages in 98ms                               
 + asttokens==3.0.0
 + comm==0.2.3
 + debugpy==1.8.17
 + decorator==5.2.1
 + executing==2.2.1
 + ipykernel==7.0.1
 + ipython==9.6.0
 + ipython-pygments-lexers==1.1.1
 + jedi==0.19.2
 + jupyter-client==8.6.3
 + jupyter-core==5.9.1
 + matplotlib-inline==0.1.7
 + nest-asyncio==1.6.0
 + numpy==2.3.4
 + packaging==25.0
 + pandas==2.3.3
 + parso==0.8.5
 + pexpect==4.9.0
 + platformdirs==4.5.0
 + prompt-toolkit==3.0.52
 + psutil==7.1.1
 + ptyprocess==0.7.0
 + pure-eval==0.2.3
 + pygments==2.19.2
 + pyparsing==3.2.5
 + python-dateutil==2.9.0.post0
 + pytz==2025.2
 + pyzmq==27.1.0
 + rdflib==7.2.1
 + six==1.17.0
 + stack-data==0.6.3
 + tentris==0.19.11b0
 + tornado==6.5.2
 + traitlets==5.14.3
 + tzdata==2025.2
 + wcwidth==0.2.14
Using Python 3.12.3 environment at: /home/rmfranken/open-pulse-quickstart/.venv


### Step 2: Imports and Configuration

This cell imports the libraries and sets up our connection variables.

In [8]:
import rdflib
import pandas as pd
import tentris # Required to register the Tentris store
from tentris import TentrisHTTPStore
from IPython.display import display, Markdown

# --- 💡 IMPORTANT 💡 ---
# This is the ONLY line you need to change.
# Set this to the server IP address provided by the hackathon organizers.
ENDPOINT_URL = "http://128.178.219.51:7502"
# ------------------------

# We will create our main 'graph' object in the next step
graph = None

### Step 3: Connect to the Server

This cell creates the `rdflib.Graph` object. It uses the `TentrisHTTPStore`, which is optimized to work with the Tentris server.

**Note:** This cell also runs a test query (`ASK { ?s ?p ?o }`) to make sure the server is reachable. If this cell fails, please double-check the `ENDPOINT_URL` you set in Step 2.

In [9]:
display(Markdown(f"🚀 Connecting to server at `{ENDPOINT_URL}`..."))

try:
    # Initialize the TentrisHTTPStore with the base endpoint URL.
    # This client is smart and knows how to find the /sparql and /stream endpoints.
    store = TentrisHTTPStore(ENDPOINT_URL)
    
    # Create the graph object
    graph = rdflib.Graph(store)
    
    # Run a simple test query to confirm the connection
    # This will raise an error if the server is unreachable
    graph.query("ASK { ?s ?p ?o }")
    
    display(Markdown("✅ **Connection successful!** The database is ready to be queried."))
    
except Exception as e:
    display(Markdown(f"❌ **Connection Failed:** Could not connect to the server.\n\n*Details: {e}*\n\nPlease check the `ENDPOINT_URL` in Step 2 and ensure the server is running."))
    graph = None # Ensure graph is None if connection fails

🚀 Connecting to server at `http://128.178.219.51:7502`...

✅ **Connection successful!** The database is ready to be queried.

### Step 4: Run Example Queries

Now you're ready to go! The `graph` object is your gateway to the database.

Here are a few examples to get you started. You can (and should!) modify these and create new cells to write your own.

In [11]:
# Example 1: Count all triples in the database
# This is a good way to see how much data you're working with.

if graph:
    query_str_count = "SELECT (COUNT(*) AS ?totalTriples) WHERE { ?s ?p ?o }"
    
    display(Markdown("**Running query:** Counting all triples..."))
    
    results = graph.query(query_str_count)
    
    for row in results:
        display(Markdown(f"Total Triples in Database: **{row.totalTriples}**"))
else:
    display(Markdown("⚠️ Skipping query: Database not connected."))

**Running query:** Counting all triples...

Total Triples in Database: **2000002**

In [12]:
# Example 2: Show 10 triples as a table
# We can use pandas to display the results in a nice table.

if graph:
    query_str_limit = "SELECT * WHERE { ?s ?p ?o } LIMIT 10"
    
    display(Markdown("**Running query:** Getting 10 triples..."))
    
    # Run the query
    results = graph.query(query_str_limit)
    
    # Convert results to a list of dictionaries
    results_list = [row.asdict() for row in results]
    
    # Load into a pandas DataFrame and display
    df = pd.DataFrame(results_list)
    display(df)
    
else:
    display(Markdown("⚠️ Skipping query: Database not connected."))

**Running query:** Getting 10 triples...

,s,p,o
0,http://example.org/software/81257,http://schema.org/contentUrl,http://example.org/downloads/software81257.zip
1,http://example.org/software/81257,http://schema.org/datePublished,2007-06-02
2,http://example.org/software/81257,http://schema.org/url,http://example.org/software/81257/
3,http://example.org/software/81257,http://schema.org/dateCreated,2007-06-02
4,http://example.org/software/81257,http://schema.org/license,https://spdx.org/licenses/MIT.html
5,http://example.org/software/81257,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/SoftwareSourceCode
6,http://example.org/software/81257,http://schema.org/name,Example Software 81257
7,http://example.org/software/81257,https://open-pulse.epfl.ch/ontology#repository...,https://open-pulse.epfl.ch/ontology#Software
8,http://example.org/software/81257,http://schema.org/codeRepository,http://github.com/exampleorg/repo81257
9,http://example.org/software/81257,http://schema.org/programmingLanguage,R


### Step 5: Your Turn! (Happy Hacking)

This is your canvas. Create new code cells below this one and start building your project.

**Tip:** Don't forget to add `LIMIT 10` to your queries while you are exploring, so you don't accidentally try to print a million rows!